<a href="https://colab.research.google.com/github/AndreyLFR/test/blob/main/Keras_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import keras
from keras import layers

использую встроенный данные

In [18]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

нормализация данных от 0 до 1

In [19]:
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


Преобразую target в массив, каждый из 10 элементов с одной 1 и нулями

In [20]:
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Определяем параметры модели

In [21]:
input_shape = (28, 28, 1)
hidden_layers = 50

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(hidden_layers, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

Создаю модель

In [22]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/15
469/469 [==============================] - 47s 98ms/step - loss: 0.2782 - accuracy: 0.9148
Epoch 2/15
469/469 [==============================] - 50s 107ms/step - loss: 0.0891 - accuracy: 0.9729
Epoch 3/15
469/469 [==============================] - 45s 97ms/step - loss: 0.0682 - accuracy: 0.9787
Epoch 4/15
469/469 [==============================] - 48s 103ms/step - loss: 0.0570 - accuracy: 0.9825
Epoch 5/15
469/469 [==============================] - 49s 105ms/step - loss: 0.0494 - accuracy: 0.9839
Epoch 6/15
469/469 [==============================] - 44s 94ms/step - loss: 0.0443 - accuracy: 0.9860
Epoch 7/15
469/469 [==============================] - 46s 97ms/step - loss: 0.0382 - accuracy: 0.9879
Epoch 8/15
469/469 [==============================] - 45s 95ms/step - loss: 0.0371 - accuracy: 0.9880
Epoch 9/15
469/469 [==============================] - 46s 99ms/step - loss: 0.0329 - accuracy: 0.9897
Epoch 10/15
469/469 [==============================] - 44s 94ms/step - loss: 0.

Результаты обучения

In [23]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.02289840765297413
Test accuracy: 0.9919000267982483


Удалось повысить точность предсказаний до 99% на тестовых данных. Платой за повышение точности служит увеличение стоимости расчетов

Подгружаю нарисованную цифру

In [47]:
from scipy.ndimage.measurements import center_of_mass
import math
import cv2
import numpy as np

def getBestShift(img):
    cy,cx = center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

def rec_digit(img_path):
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  gray = 255-img
  # применяем пороговую обработку
  (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  # удаляем нулевые строки и столбцы
  while np.sum(gray[0]) == 0:
    gray = gray[1:]
  while np.sum(gray[:,0]) == 0:
    gray = np.delete(gray,0,1)
  while np.sum(gray[-1]) == 0:
    gray = gray[:-1]
  while np.sum(gray[:,-1]) == 0:
    gray = np.delete(gray,-1,1)
  rows,cols = gray.shape

  # изменяем размер, чтобы помещалось в box 20x20 пикселей
  if rows > cols:
    factor = 20.0/rows
    rows = 20
    cols = int(round(cols*factor))
    gray = cv2.resize(gray, (cols,rows))
  else:
    factor = 20.0/cols
    cols = 20
    rows = int(round(rows*factor))
    gray = cv2.resize(gray, (cols, rows))

  # расширяем до размера 28x28
  colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
  rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
  gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')

  # сдвигаем центр масс
  shiftx,shifty = getBestShift(gray)
  shifted = shift(gray,shiftx,shifty)
  gray = shifted

  cv2.imwrite('gray'+ img_path, gray)
  img = gray / 255.0
  img = np.array(img).reshape(-1, 28, 28, 1)
  out = str(np.argmax(model.predict(img)))
  return out

img = rec_digit('/content/2024-02-29 15.54.10.jpg')
print(img)


1/1 [==============================] - 0s 37ms/step
7


<ipython-input-47-1d5f457db001>:1: DeprecationWarning: Please use `center_of_mass` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  from scipy.ndimage.measurements import center_of_mass
